<a href="https://colab.research.google.com/github/emilynairnn/msci598_final_project/blob/main/LSTM_with_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup


In [ ]:
pip install keras_self_attention

  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18912 sha256=825abd7b01ed4da8174b36d6a994ac5df28119a27afa2601ee8ca649b95fe25a
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-self-attention


In [ ]:
import pandas as pd
from google.colab import drive 
drive.mount('/content/gdrive')

import sys
sys.path.insert(0,'gdrive/My Drive/MSCI598 Project')
from utils.score import report_score, LABELS, score_submission

import numpy as np
from sklearn import feature_extraction
# from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer, TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras import Sequential, regularizers
from keras import optimizers
from keras import backend as K
from keras import initializers, constraints, regularizers
from keras.layers import Reshape, Dot, Concatenate, Input, Embedding, Dropout, Dense, LSTM, Bidirectional, Activation, BatchNormalization, Flatten
from keras.utils import np_utils
from keras.models import Model
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from scipy import sparse
from scipy.sparse import csr_matrix
import re
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from keras_self_attention import SeqSelfAttention
import keras
import tensorflow as tf
from keras_self_attention import SeqSelfAttention
from keras import optimizers



Mounted at /content/gdrive


In [ ]:
dir = 'gdrive/My Drive/MSCI598 Project/GoogleNews-vectors-negative300.bin'
wv = gensim.models.KeyedVectors.load_word2vec_format(dir, binary=True)

In [ ]:
MAX_FEATURES = 5000
BATCH_SIZE = 128
N_EPOCHS = 10
MAX_VOCAB_SIZE = 30000
MAX_SENT_LEN = 2
EMBEDDING_DIM = 300
LSTM_DIM = 128

# lstm

preprocessing code 

In [ ]:
def clean(content):
  list = " ".join(re.findall(r'\w+', content, flags=re.UNICODE)).lower()
  return " ".join([word for word in content.split(" ") if word not in feature_extraction.text.ENGLISH_STOP_WORDS])

train_bodies = pd.read_csv('gdrive/My Drive/MSCI598 Project/fnc-1-master/train_bodies.csv')
train_stances = pd.read_csv('gdrive/My Drive/MSCI598 Project/fnc-1-master/train_stances.csv')
train_combined = train_stances.join(train_bodies.set_index('Body ID'), on='Body ID')

test_bodies = pd.read_csv('gdrive/My Drive/MSCI598 Project/fnc-1-master/competition_test_bodies.csv')
test_stances = pd.read_csv('gdrive/My Drive/MSCI598 Project/fnc-1-master/competition_test_stances.csv')
test_combined = test_stances.join(test_bodies.set_index('Body ID'), on='Body ID')

train_headlines_clean = [clean(headline) for headline in train_combined['Headline']]
train_bodies_clean = [clean(body) for body in train_combined['articleBody']]

test_headlines_clean = [clean(headline) for headline in test_combined['Headline']]
test_bodies_clean = [clean(body) for body in test_combined['articleBody']]


In [ ]:
train_word_seq_headlines = [text_to_word_sequence(headline) for headline in train_headlines_clean]
train_word_seq_bodies = [text_to_word_sequence(body) for body in train_bodies_clean]
test_word_seq_headlines = [text_to_word_sequence(headline) for headline in test_headlines_clean]
test_word_seq_bodies = [text_to_word_sequence(body) for body in test_bodies_clean]

train_word_seq = [None]*len(train_word_seq_headlines)
for i in range(len(train_word_seq_headlines)):
  train_word_seq[i] = train_word_seq_headlines[i] + train_word_seq_bodies[i]

test_word_seq = [None]*len(test_word_seq_headlines)
for i in range(len(test_word_seq_headlines)):
  test_word_seq[i] = test_word_seq_headlines[i] + test_word_seq_bodies[i]


In [ ]:
#preprocessing
word_seq = train_word_seq + test_word_seq
tokenizer = Tokenizer(num_words = MAX_VOCAB_SIZE)
tokenizer.fit_on_texts([' '.join(seq[:MAX_SENT_LEN]) for seq in word_seq])

X_train = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in train_word_seq])
X_train = pad_sequences(X_train, maxlen = MAX_SENT_LEN, padding = 'post',truncating = 'post')
label_encoder_train = LabelEncoder().fit_transform(train_combined['Stance'])
Y_train = np_utils.to_categorical(label_encoder_train, num_classes = 4)

X_test = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in test_word_seq])
X_test = pad_sequences(X_test, maxlen = MAX_SENT_LEN, padding = 'post',truncating = 'post')
label_encoder_test = LabelEncoder().fit_transform(test_combined['Stance'])
Y_test = np_utils.to_categorical(label_encoder_test, num_classes = 4)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, random_state=10, test_size=0.1)



[[140]
 [ 14]
 [174]
 [ 98]
 [520]
 [279]
 [465]
 [212]
 [ 53]
 [505]
 [  8]
 [260]
 [ 12]
 [390]
 [637]
 [  6]
 [ 12]
 [348]
 [ 34]]


In [ ]:
import keras
import tensorflow as tf
from keras_self_attention import SeqSelfAttention
from keras import optimizers

model = Sequential()
model.add(Embedding(MAX_FEATURES, EMBEDDING_DIM, input_length=4))
model.add(Bidirectional(LSTM(units=128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(Flatten())
model.add(Dense(4, activation='sigmoid'))

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    # loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    #loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 300)            1500000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 4, 256)           439296    
 nal)                                                            
                                                                 
 seq_self_attention_1 (SeqSe  (None, 4, 256)           16449     
 lfAttention)                                                    
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 4)                 4100      
                                                                 
Total params: 1,959,845
Trainable params: 1,959,845
No

In [ ]:
model.fit(X_train,Y_train,
          batch_size = BATCH_SIZE,
          epochs = N_EPOCHS,
          validation_data=(X_val,Y_val))

Epoch 1/10
352/352 [==============================] - 55s 116ms/step - loss: 0.9494 - accuracy: 0.7201 - val_loss: 0.7884 - val_accuracy: 0.7291
Epoch 2/10
352/352 [==============================] - 40s 113ms/step - loss: 0.7650 - accuracy: 0.7316 - val_loss: 0.7659 - val_accuracy: 0.7291
Epoch 3/10
352/352 [==============================] - 41s 117ms/step - loss: 0.7256 - accuracy: 0.7314 - val_loss: 0.7332 - val_accuracy: 0.7285
Epoch 4/10
352/352 [==============================] - 40s 115ms/step - loss: 0.7005 - accuracy: 0.7309 - val_loss: 0.7252 - val_accuracy: 0.7263
Epoch 5/10
352/352 [==============================] - 40s 112ms/step - loss: 0.6896 - accuracy: 0.7311 - val_loss: 0.7222 - val_accuracy: 0.7259
Epoch 6/10
352/352 [==============================] - 40s 114ms/step - loss: 0.6834 - accuracy: 0.7310 - val_loss: 0.7212 - val_accuracy: 0.7249
Epoch 7/10
352/352 [==============================] - 40s 115ms/step - loss: 0.6798 - accuracy: 0.7303 - val_loss: 0.7216 - val_ac

In [ ]:
model.evaluate(X_test, Y_test,
               batch_size = BATCH_SIZE)

predicted = [LABELS[np.argmax(i)] for i in model.predict(X_test)]
actual = [LABELS[np.argmax(i)] for i in Y_test]
np.savetxt("answer.csv", predicted, delimiter=",", fmt='%s')
report_score(actual,predicted)

199/199 [==============================] - 4s 18ms/step - loss: 0.9005 - accuracy: 0.7146
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |     7     |     0     |     6     |   1890    |
-------------------------------------------------------------
| disagree  |     0     |     0     |     1     |    696    |
-------------------------------------------------------------
|  discuss  |    12     |     9     |    44     |   4399    |
-------------------------------------------------------------
| unrelated |    70     |    20     |    151    |   18108   |
-------------------------------------------------------------
Score: 4585.0 out of 11651.25	(39.35200085827701%)


39.35200085827701

In [ ]:
from google.colab import files
df.to_csv('output.csv', index=False, encoding = 'utf-8') 
files.download('answer.csv')